# Generalized Network Analysis Tutorial - Step 1

The Network Analysis Tutorial is part of the work entitled **Generalized correlation-based dynamical network analysis: a new high-performance approach for identifying allosteric communications in molecular dynamics trajectories**, by Marcelo C. R. Melo, Rafael C. Bernardi, Cesar de la Fuente-Nunez, and Zaida Luthey-Schulten. For more information see http://faculty.scs.illinois.edu/schulten/. 

In this tutorial, we will use the Dynamic Network Analysis python package to explore the interactions between the OMP decarboxylase enzyme and its substrate, identifying clusters of amino acid residues that form domains, and active site residues that are important for binding and enzymatic activity.

The tutorial is divided in two jupyter notebooks. In this notebook, **Step 1**, we will analyze the MD trajectory for the OMP decarboxylase system and generate the network data used for analysis. 

The trajectory files have approximately 500MB in size, and must be downloaded [from this link](http://www.ks.uiuc.edu/~rcbernardi/NetworkAnalysis/DynamicNetworkAnalysis_MDdata.tar.gz) and placed in the *TutorialData* folder.

In the accompanying notebook, **Step 2**, we will load the data generated in the Step 1, and create analysis plots and visualizations of the network. 

In **Step 3**, which is outside of the jupyter notebook environment, we will produce high-quality renderings of the network using the popular visualization software VMD.

In [ ]:
# Load the python package
import os
import dynetan
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
import argparse
from operator import itemgetter

In [ ]:
# Create the object that processes MD trajectories.
dnap = DNAproc()

In [ ]:
bPythonExport = False

In [ ]:
mapResidueNames={'ALA':'A','CYS':'C','ASP':'D','GLU':'E','PHE':'F',
                 'GLY':'G','HIS':'H','HSD':'H','HSE':'H','ILE':'I','LYS':'K','LEU':'L',
                 'MET':'M','ASN':'N','PRO':'P','GLN':'Q','ARG':'R',
                 'SER':'S','THR':'T','VAL':'V','TRP':'W','TYR':'Y',
                 'MG':'Mg','ATP':'Atp','POPC':'Popc','SOL':'h2o'}
def name_node(dnap, node):
    #i=dnap.nodesAtmSel[node].index
    resname=dnap.nodesAtmSel[node].resname ; resid=dnap.nodesAtmSel[node].resid
    return "%s%s" % (mapResidueNames[resname], resid)

def clarify_duplicate_nodes(dictNames, dictSuffix):
    """
    From two dicts with the same keys, add the respective suffix to all keys in the former that possess duplicate values.
    """
    from itertools import chain
    dictRev = {}
    for k, v in dictNames.items():
        dictRev.setdefault(v, set()).add(k)
    setDuplicateKeys = set(chain.from_iterable( v for k, v in dictRev.items() if len(v) > 1))
    
    for k in setDuplicateKeys:
        dictNames[k] = dictNames[k]+"_"+dictSuffix[k]
    return dictNames

# Node Groups

In Network Analysis, each residue is represented by one or more "nodes", serving as proxies for groups of atoms form the original residue (Figure 1). This approach lowers computational cost and noise.

For our purposes, we treat this as a coarse-graining procedure such that large molecules like ATP and POPC will gain multiple nodes to better distinguish between connections.

In [ ]:
if bPythonExport:
    parser = argparse.ArgumentParser(description='Process trajactories for Dynamic Network Analysis.'
                                                 'Equivalent to Step 1 of the tutorial without using Jupyter.',
                                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-o', '--output', type=str, dest='outputPrefix', default=None,
                        help='Prefix used for all output files.')    
    parser.add_argument('--top', type=str, dest='fileTopology', default='seg.psf',
                        help='The name of a suitable file for MDAnalysis to generate a topology.')
    parser.add_argument('fileTrajectories', type=str, metavar='N', nargs='+', default='sum.xtc',
                        help='One or more trajectory files for MDAnalysis to read in. All frames are combined together.')
    parser.add_argument('-n', '--num_windows', type=int, dest='numWinds', default=None,
                        help='[Opt.] Split the total trajectory frames into N windows. By default equal to the number of files read.')
    parser.add_argument('-f', '--num_frames', type=int, dest='numFrames', default=100,
                        help='Number of frames to read for each window. Total number of frames will then be nWind*nFrames. '
                       'This is computed by DNA as the total frames divided evenly.')
#    = = = Very inconvenient to do within MDAnalysis. Outside design doc.
#    parser.add_argument('--no_span', dest='bNoSpan', action='store_true',
#                        help='When the number of frames is given, prevent windows from spanning across multiple trajectory files. '
#                       'This will cause remainder frames to be discarded.')
    parser.add_argument('--cpus', type=int, dest='nCPUs', default=1,
                        help='Set number of processes to enable partial parallelisation.')
    #   =============== Analysis parameters=
    parser.add_argument('--exclude_solvent', action='store_true', dest='bExcludeSolvent',
                        help='Boolean to exclude potential solvent contacts in the analysis.')
    parser.add_argument('--cutoff', type=float, dest='distCutoff', default=4.5,
                        help='Distance in Angstroms to consider two nodes as being in contact.'
                             'Calculated as the nearest distance between all atoms of the node.')
    parser.add_argument('--persistence', type=float, dest='ratioContactPersistence', default=0.75,
                        help='Minimum portion of frames for DNA to consider two nodes to be in sufficient contact,'
                             'such that it will include this node pair as an edge in the output graph.')
    args = parser.parse_args()

    numCoresAvailable=args.nCPUs

    # = = = Analysis Configurations = = =
    # Cutoff for contact map (In Angstroms)
    cutoffDist = args.distCutoff
    # Minimum contact persistance (In ratio of total trajectory frames)
    contactPersistence = args.ratioContactPersistence
    # Inclusion of solvent
    bIncludeSolvent = not args.bExcludeSolvent

    topFile = args.fileTopology
    trjFiles = args.fileTrajectories

    if args.outputPrefix is None:
        pathToData = "./results" % (allele, temperature)
        fileNameRoot = "analysis"
    else:
        pathToData, fileNameRoot = os.path.split(args.outputPrefix)
        
    fullPathRoot = os.path.join(pathToData, fileNameRoot)

    # Number of windows created from full simulation.
    if args.numWinds is not None:
        numWinds = args.numWinds
    else:
        numWinds = len(args.fileTrajectories)
        
    # Sampled frames per window
    numSampledFrames = args.numFrames

In [ ]:
if not bPythonExport:
    numCoresAvailable=6
    systemExample="Ubq"    
    if systemExample is "UbqCHARMM":
        # apo1 apo2 apo3
        state="apo1" ; workDir = "./apo-md01"
        topFile = "./tops/prot-segids.pdb"
        trjFiles= os.path.join(workDir, "prot-masses.xtc" )
        numWinds = 5 ; numSampledFrames = 1000
        pathToData = "./dynetan"
        fileNameRoot = "%s_%ix%i" % (state, numWinds, numSampledFrames)
        fullPathRoot = os.path.join(pathToData, fileNameRoot)     
    elif systemExample is "UbqAthi":
        # UbqI13V  UbqI23A  UbqI30A  UbqL43A  UbqL67A  UbqL69A  UbqV17A  UbqWT
        state="UbqV17A" ; workDir = "./%s" % state
        topFile = os.path.join(workDir, "reference.pdb" )
        trjFiles= os.path.join(workDir, "traj-1ns.xtc" )
        numWinds = 5 ; numSampledFrames = 200
        pathToData = "./dynetan"
        fileNameRoot = "%s_%ix%i" % (state, numWinds, numSampledFrames)
        fullPathRoot = os.path.join(pathToData, fileNameRoot)
    elif systemExample is "periplasmic":
        # apo holo-leu
        state="apo" ; workDir = "./apo"
        topFile = "./apo/tops/prot-segids.pdb"
        trjFiles= os.path.join(workDir, "protcent.xtc" )
        numWinds = 5 ; numSampledFrames = 200
        pathToData = "./dynetan"
        fileNameRoot = "%s_%ix%i" % (state, numWinds, numSampledFrames)
        fullPathRoot = os.path.join(pathToData, fileNameRoot)        
    elif systemExample is "CFTR":
        # Define mutant file IO locations. wt, P67L, E56K, R75Q, dF508 , S945L
        allele="wt" ; temperature="310K" ; nRepl=6
        workDir = "./trajectories/%s/%s/" % (allele, temperature)
        topFile = os.path.join(workDir, "1/clustered_d3.5_r0.50.pdb")
        trjFiles=[]
        for i in range(1,nRepl+1):
            trjFiles.append(os.path.join(workDir, "%i/clustered_d3.5_r0.50.xtc" %i))
        pathToData = "./results/%s/%s/" % (allele, temperature)
        fileNameRoot = "1to%i" % nRepl
        fullPathRoot = os.path.join(pathToData, fileNameRoot)        
        numWinds = 5 ; numSampledFrames = 200
        else:
            Insert_Interrupt()

In [ ]:
# = = = Change to the relevant work folder
if not bPythonExport:
    if systemExample is "UbqCHARMM":
        %cd /home/zharmad/host/projects/Ubq-md
    elif systemExample is "UbqAthi":
        %cd /home/zharmad/host/shared-colleague/Ubq-2017
    elif systemExample is "periplasmic":
        %cd /home/zharmad/projects/periplasmic/leucine-binding_protein
    elif systemExample is "CFTR":
        %cd /home/zharmad/projects/cftr/DyNetAn

In [ ]:
if numCoresAvailable>1:
    strBackend="openmp"
else:
    strBackend="serial"

if not os.path.exists(pathToData):
    os.makedirs(pathToData)

In [ ]:
# Interface selection Is currently pre-set and not interchangable as we don't currently use it.

# Domain N-lobe: 1 to 120 275 to 329
# Domain loop.: 251 to 274
# Domain C-lobe: 121 to 250 330 to 346
# Half-site A (5 Angs. to LEU ligand): 15 18 77 78 79 80 100 101 102 103 276
# Half-site B (5 Angs. to LEU ligand): 118 121 124 150 202 226 227
#if state != "apo":
#    seltextInterfaceA="resid 15 18 77 78 79 80 100 101 102 103 118 121 124 150 202 226 227 276"
#    seltextInterfaceB="resid 347"
#else:
seltextInterfaceA="resid 30 43"
seltextInterfaceB="resid 67 69"

In [ ]:
# = = Define selections = = 

# = = Following the NAMD/VMD system, DynaNet uses a segment syntax instead of chain syntax to classify atoms.
#     Thus, segments will need to be defined in the topology file.

# = = Segment IDs for regions that will be studied.
segIDs = ["UBQ"]
    
h2oName = ["SOL"]

# Number of sampled frames for automatic selection of solvent and ions.
# numAutoFrames = numSampledFrames*numWinds

# Network Analysis will make one node per protein residue (in the alpha carbon)
# For all other residues, the user must specify atom(s) that will represent a node.
# ...
# We also need to know the heavy atoms that compose each node group.

customResNodes = {}
usrNodeGroups = {}

# Cater to HSD and HSE in the CHARMM forcefield.
# Lip: Uses POPC as a reference. Node designations include:
#      ...zwitterion centers, ester centers, and two additional carbon centers for each tail capturing ~7 atoms.
#      ...heavy atom assignment based on charge groups in CHARMMM36
#      ...a PSFGEN error means that the naming of carbons C216 -> 6C12.
#customResNodes["HSD"] = ["CA"]
#usrNodeGroups["HSD"] = {}
#usrNodeGroups["HSD"]["CA"] = set("N CA CB ND1 CG CE1 NE2 CD2 C O".split())
#customResNodes["HSE"] = ["CA"]
#usrNodeGroups["HSE"] = {}
#usrNodeGroups["HSE"]["CA"] = set("N CA CB ND1 CG CE1 NE2 CD2 C O".split())

customResNodes["K"] = ["K"]
usrNodeGroups["K"] = {}
usrNodeGroups["K"]["K"] = set("K".split())

customResNodes["CL"] = ["CL"]
usrNodeGroups["CL"] = {}
usrNodeGroups["CL"]["CL"] = set("CL".split())

customResNodes["SOL"] = ["OW"]
usrNodeGroups["SOL"] = {}
usrNodeGroups["SOL"]["OW"] = set("OW HW1 HW2".split())

#################################
### Extra configuration

# Cutoff for contact map (In Angstroms)
cutoffDist = 4.5

# Minimum contact persistance (In ratio of total trajectory frames)
contactPersistence = 0.75

In [ ]:
#################################
### Load info to object

dnap.setNumWinds(numWinds)
dnap.setNumSampledFrames(numSampledFrames)
dnap.setCutoffDist(cutoffDist)
dnap.setContactPersistence(contactPersistence)
dnap.seth2oName(h2oName)
dnap.setSegIDs(segIDs)

dnap.setCustomResNodes(customResNodes)
dnap.setUsrNodeGroups(usrNodeGroups)

# Load the trajectory

Our Generalized Network Analysis leverages the MDAnalysis package to create a *universe* that contains all the trajectory and system information.

In [ ]:
dnap.loadSystem(topFile,trjFiles)

### Checks segments and residue names

This is important to know if there are residues in the structure that we didn't know of, and need to be addresssed so that network analysis can create nodes in all selected residues.

In [ ]:
dnap.checkSystem()

In [ ]:
# More checks?
# %whos to get all variables

### Automatically identify crystallographic waters and ions.

In this section we identify all residues which will be checked for connectivity with selected segments.

- First, we define if all solvent molecules will be checked, or if just ions, ligands, lipids, and other molecules will be checked.

- Second, we sample a small set of frames from the trajectory to select likely residues, then we check all trajectory to see if they are closer than the cutoff distance for at least x% of the simulation (where x is the "contact persistence" fraction of the trajectory).

- Third, we load the trajectory of the relevant atoms to memory to speed up the analysis

In [ ]:
dnap.selectSystem(withSolvent=False)

### Prepare network representation of the system

Here we check that we know how to treat all types of residues in the final selection. Every residue will generate one or more nodes in the final network. Then we store the groups of atoms that define each node.

In [ ]:
# dnap.getU().residues[300]
# dnap.getU().residues[300].resname
# 'C13' in set.union(*dnap.resNodeGroups['POPC'].values())

In [ ]:
dnap.prepareNetwork()

## Align the trajectory based on selected segments

We align the trajectory to its first frame using heavy atoms (non-hydrogen) from the selected segments. In the process, we also transfer the trajectory to the computer memory, so that future analysis and manipulations are completed faster.

In [ ]:
# If your system is too large, you can turn off the "in memory" option, at a cost for performance.
dnap.alignTraj(inMemory=True)

### Select residues that are closer than 4.5A for more than 75% of simulation

Creates an N-by-N matrix for all N nodes in the selected region, and automatically selected nodes (ions, solvent).

The following cell defines efficient functions to run the analysis and create a contact matrix. We leverage both MDAnalysis parallel contact detection tools, as well as accelerated Numba and Cython function. After creating the contact matrix, we remove any automatically selected nodes that have insuficient persistance, and filter the contacts by (optionally) removing contacts between nodes in consecutive residues in a protein or nucleic chain.


**Attention** For every time you start this Jupyter Notebook, the first time you execute this function may take significanlty longer (several seconds) to start. This is because we use *Cython* and *Numba* to compile functions "on-demand", and a new compilation may be necessary after the notebook is re-started.

In [ ]:
# To speed-up the contact matrix calculation, a larger stride can be selected, at a cost for precision.
dnap.findContacts(stride=1)

### Removing contacts between nodes in the same residue.

The following function guarantees that there will be no "self-contacts" (contacts between a node and itself), and gives you the opportunity to remove contacts between nodes in consecutive residues (such as sequential amino acids in the same chain, removing back-bone interactions). 

The function also removes nodes that are never in contact with any other node in the system (such as the ends of flexible chains, or residues in flexible loops). This will automatically update the MDanalysis universe and related network informatio, such as number of nodes and atom-to-node mappings.

In [ ]:
dnap.filterContacts(notSameRes=True, notConsecutiveRes=False, removeIsolatedNodes=True)

## Calculate Generalized Correlation with Python/Numba

In [ ]:
# We can calculate generalized correlaions in parallel using Python's multiprocessing package.
dnap.calcCor(ncores=numCoresAvailable)

## Calculate cartesian distances between all nodes in the selected system.

Here, we will calculate the **shortest** distance between atoms in all pairs of nodes. It is similar to the contact matrix calculation, but we check all distances and keep the shortest one to use in our analysis.

In [ ]:
print( dnap.atomToNode.shape )

In [ ]:
# We can leverage MDanalysis parallelization options with backend="serial" or backend="openmp".
# For very small systems, the serial can be faster!
dnap.calcCartesian(backend=strBackend)

# Network Calculations
Create a graph from our correlation matrix. Different properties are calculated:

*Density* measures how connected the graph is compared to how connected it *could* be. It is the ratio between edges in the graph over all possible edges between all pairs of nodes.

*Transitivity* maesures the triadic closure, comparing present triangles to possible triangles. In a triangle, if A is connected to B, and B connected to C, then A is connected to C.

*Degree* measures the number of connections a node has.

(Reference)[1]

[1]:https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python#advanced-networkx-community-detection-with-modularity

In [ ]:
dnap.calcGraphInfo()

In [ ]:
# Basic information of the network as interpreted as a graph.
print( nx.info(dnap.nxGraphs[0]))

In [ ]:
# Both density and transitivity are scaled from 0 to 1
for win in range(dnap.numWinds):
    print("----- Window {} -----".format(win))
    print("Density:", round( nx.density(dnap.nxGraphs[win]), 4) )
    print("Transitivity:", round( nx.transitivity(dnap.nxGraphs[win]), 4) )
    print()

In [ ]:
from operator import itemgetter

# We can check the nodes that have the most connections in each window.
for win in range(dnap.numWinds):
    print("----- Window {} -----".format(win))
    
    sorted_degree = sorted(dnap.getDegreeDict(win).items(), key=itemgetter(1), reverse=True)
    
    print("Top 10 nodes by degree: [node --> degree : selection]")
    for n,d in sorted_degree[:5]:
        print("{0:>4} --> {1:>2} : {2}".format(n, d, getSelFromNode(n, dnap.nodesAtmSel)))
    
    print()

## Calculate optimal paths
We choose the Floyd Warshall algorithm[1]. This uses the **correlations as weights** to calculate network distances and shortest distances.

[1]:https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.dense.floyd_warshall.html?highlight=warshall#networkx.algorithms.shortest_paths.dense.floyd_warshall

In [ ]:
dnap.calcOptPaths(ncores=numCoresAvailable)

## Calculate betweenness

We calculate both betweenness centrality[1] for edges and eigenvector centrality[2] for nodes.

[1]:https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html?highlight=betweenness#networkx.algorithms.centrality.edge_betweenness_centrality
[2]:https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.eigenvector_centrality.html

In [ ]:
dnap.calcBetween(ncores=numCoresAvailable)

In [ ]:
from itertools import islice

# Pairs of nodes with highest Betweeness values, compared to their correlation values (in Window 0)
for k,v in islice(dnap.btws[0].items(),5):
    print("Nodes {} have betweenes {} and correlation {}.".format(k, 
                                                                  round(v,3), 
                                                                  round(dnap.corrMatAll[0, k[0], k[1]], 3) ) )

Turn to node centrality instead of edge centrality:

In [ ]:
dnap.calcEigenCentral()

### Calculate communities



Using **Louvain heuristices** is feasible. 
This method also maximizes the modularity of the network.

http://iopscience.iop.org/article/10.1088/1742-5468/2008/10/P10008/meta

In [ ]:
dnap.calcCommunities()

In [ ]:
# Sort communities based on number of nodes
for comIndx in dnap.nodesComm[0]["commOrderSize"]:
    print("Modularity Class {0:>2}: {1:>3} nodes.".format(comIndx, len(dnap.nodesComm[0]["commNodes"][comIndx])))

In [ ]:
# Sort communities based on the node with highest eigenvector centrality
for comIndx in dnap.nodesComm[0]["commOrderEigenCentr"]:
    print("Modularity Class {0} ({1} nodes) Sorted by Eigenvector Centrality:".format(
                                                                    comIndx, 
                                                                len(dnap.nodesComm[0]["commNodes"][comIndx])))
    for node in dnap.nodesComm[0]["commNodes"][comIndx][:5]:
        print("Name: {0:>4} | Degree: {1:>2} | Eigenvector Centrality: {2}".format(
            node, dnap.nxGraphs[win].nodes[node]['degree'], dnap.nxGraphs[win].nodes[node]['eigenvector']))
    print()

# Process Interface Residues

We now find all nodes that are close to both selections chosen by the user. That may include amino acids in the interface, as well as ligands, waters and ions.

In [ ]:
dnap.interfaceAnalysis(selAstr=seltextInterfaceA, selBstr=seltextInterfaceB)

Compute additional graph properties here to save some time for the analysis in Step 2.

In [ ]:
# Note: btws is the same as the results derived from nx.edge_betweenness_centrality()
for w in range(numWinds):
    G = dnap.nxGraphs[w]

    # Transfer edge-betweenness to graph.
    for u,v in dnap.btws[w]:
        G.edges[u,v]['btws']=dnap.btws[w][u,v]
    # Note: edge-betweeness weighted clustering coefficient. 
    c = nx.clustering(G, weight='btws')
    for x in range(dnap.numNodes):
        G.nodes[x]['bwcc']=c[x]
    # Node betweenness centrality as an alternative to eigenvector centrality.
    c = nx.betweenness_centrality(dnap.nxGraphs[w])
    for x in range(dnap.numNodes):
        dnap.nxGraphs[w].nodes[x]['btws']=c[x]

    # Set the name of the node for Hover display. Append atom names to residues that have multiple nodes.
    nodeNames={} ; nodeSegIDs={} ; nodeAtomNames={}
    for x in G.nodes():
        #i=dnap.nodesAtmSel[x].index
        nodeNames[x]  = name_node(dnap, x)
        nodeSegIDs[x] = dnap.nodesAtmSel[x].segid
        nodeAtomNames[x] = dnap.nodesAtmSel[x].name
    nodeNames = clarify_duplicate_nodes( nodeNames, nodeAtomNames )
    nx.set_node_attributes(G, nodeNames, "name")
    nx.set_node_attributes(G, nodeSegIDs, "segid")

## Save data and reduced trajectory for visualization

In [ ]:
dnap.saveData(fullPathRoot)

In [ ]:
# This function will save a reduced DCD trajectory with the heavy atoms used for network analysis
# A smaller trajectory can be created by choosing a "stride" that sub-samples the original trajectory.
# This function will also produce a PDB file so that information on atoms and residues can be loaded to
#    visualization software such as VMD.

dcdstride = 1
print("We will save {} heavy atoms and {} frames.".format(dnap.workU.atoms.n_atoms, 
                                                          len(dnap.workU.trajectory[::dcdstride]) ))

In [ ]:
dnap.saveReducedTraj(fullPathRoot, stride = dcdstride)

MDAnalysis may print warnings regarding missing data fields, such as altLocs, icodes, occupancies, or tempfactor, which provide information commonly found in PDB files.
The warnings are for your information and in the context of this tutorial they are expected and do not indicate a problem.

# Analysis

The we have finished processing the trajectory and storing all related data. We can now move on to analysis of the network properties calculated here.

**All analysis code was placed in a second tutorial notebook for clarity.**


# ---- The End ----